In [ ]:
!pip install matplotlib scipy numpy pandas pillow shapely

In [ ]:
# Import required modules
import numpy as np
import matplotlib.pyplot as plt
from genetic_optimizer import GeneticOptimizer, ParallelEvaluator

# Optimization parameters
n_generations = 1
population_size = 1
n_dots = 0

# Initialize genetic algorithm optimizer
optimizer = GeneticOptimizer(
    population_size=population_size,
    n_dots=n_dots,
    x_range=(-2.4e-6, -0.1e-6),
    y_range=(-0.4e-6, 0.4e-6),
    mutation_rate=0.2,
    mutation_std=100e-9,
    crossover_rate=0.7,
    elite_fraction=0.2
)

# Initialize population
optimizer.initialize_population(seed=42)

# Create parallel evaluator (automatically detects GPUs)
evaluator = ParallelEvaluator()

print("\nStarting parallel optimization...")
print(f"Population size: {population_size}")
print(f"Number of dots: {n_dots}")
print(f"Generations: {n_generations}")

In [ ]:
# Prepare geometry PNG and regions OVF before optimization
# This creats 253 regions for exponential edge alphas (1 region saved for stripline, 1 saved for other elements e.g. magnetic dots)
import os
import numpy as np
from PIL import Image

dx = 20e-9; dy = 20e-9; dz = 10e-9
nx = 250; ny = 50; nz = 20
regions_png = 'mumax_regions.png'
geom_png = 'mumax_geometry.png'
ovf_path = 'regions_map.ovf'

# 1) Generate geometry/regions PNG via regions_generator (geometry png is required)
try:
    import regions_generator
    print("Generating geometry/regions PNGs via regions_generator...")
    regions_path, geom_path = regions_generator.generate_regions_image(
        output_path=regions_png,
        geom_path=geom_png
)
    print(f"Generated: {regions_path}, {geom_path}")
except Exception as e:
    print(f"Warning: PNG generation skipped or failed: {e}")

if not (os.path.exists(regions_png) and os.path.exists(geom_png)):
    raise FileNotFoundError("Missing 'mumax_regions.png' or 'mumax_geometry.png'.")

# 2) Build 3D regions OVF file (uniform along z) from 2D PNG
arr2d = np.array(Image.open(regions_png).convert('L'))
if arr2d.shape != (ny, nx):
    print(f"Note: resizing regions PNG from {arr2d.shape} to (ny, nx)=({ny}, {nx}) for OVF consistency")
    arr2d = np.array(Image.fromarray(arr2d).resize((nx, ny), resample=Image.NEAREST))
regions3d = np.repeat(arr2d[None, :, :], nz, axis=0).astype(np.int32)

def write_ovf_text_scalar(path, data3d, dx, dy, dz):
    nz, ny, nx = data3d.shape
    xmax = nx * dx; ymax = ny * dy; zmax = nz * dz
    with open(path, 'w') as f:
        f.write('# OOMMF OVF 2.0\n')
        f.write('# Segment count: 1\n')
        f.write('# Begin: Segment\n')
        f.write('# Begin: Header\n')
        f.write('# Title: regions\n')
        f.write('# meshtype: rectangular\n')
        f.write('# meshunit: m\n')
        f.write('# xmin: 0\n')
        f.write('# ymin: 0\n')
        f.write('# zmin: 0\n')
        f.write(f'# xmax: {xmax}\n')
        f.write(f'# ymax: {ymax}\n')
        f.write(f'# zmax: {zmax}\n')
        f.write('# valuedim: 1\n')
        f.write('# valuelabels: regions\n')
        f.write('# valueunits: 1\n')
        f.write(f'# xbase: {dx/2}\n')
        f.write(f'# ybase: {dy/2}\n')
        f.write(f'# zbase: {dz/2}\n')
        f.write(f'# xnodes: {nx}\n')
        f.write(f'# ynodes: {ny}\n')
        f.write(f'# znodes: {nz}\n')
        f.write(f'# xstepsize: {dx}\n')
        f.write(f'# ystepsize: {dy}\n')
        f.write(f'# zstepsize: {dz}\n')
        f.write('# End: Header\n')
        f.write('# Begin: Data Text\n')
        # Fortran-like flatten: x fastest, then y, then z
        for k in range(nz):
            for j in range(ny):
                for i in range(nx):
                    f.write(f'{int(data3d[k, j, i])} \n')
        f.write('# End: Data Text\n')
        f.write('# End: Segment\n')

write_ovf_text_scalar(ovf_path, regions3d, dx, dy, dz)
print(f"Wrote OVF to {ovf_path} with shape (nz, ny, nx)=({nz}, {ny}, {nx})")

In [ ]:
# Run optimization loop
# This uses 
for gen in range(n_generations):
    print(f"\n{'='*60}")
    print(f"GENERATION {gen}")
    print(f"{'='*60}")
    
    # Get current population
    population = optimizer.get_current_population()
    
    # Evaluate population in parallel across GPUs
    fitness_scores = evaluator.evaluate_population(population, gen)
    
    # Update optimizer with fitness scores
    optimizer.set_fitness_scores(fitness_scores)
    
    # Save checkpoint
    optimizer.save_checkpoint(f'checkpoint_gen{gen}.json')
    
    # Evolve to next generation (unless this is the last generation)
    if gen < n_generations - 1:
        optimizer.evolve()

print("\n" + "="*60)
print("OPTIMIZATION COMPLETE")
print("="*60)

In [ ]:
# Plot optimization progress
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Plot fitness evolution
generations = optimizer.history['generations']
best_fitness = optimizer.history['best_fitness']
mean_fitness = optimizer.history['mean_fitness']

ax1.plot(generations, best_fitness, 'o-', label='Best', linewidth=2)
ax1.plot(generations, mean_fitness, 's-', label='Mean', linewidth=2)
ax1.set_xlabel('Generation')
ax1.set_ylabel('Fitness (Selectivity Score)')
ax1.set_title('Genetic Algorithm Progress')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot best dot configuration
best_individual, best_fitness_final = optimizer.get_best_individual()
x_coords = [pos[0] * 1e6 for pos in best_individual]  # Convert to µm
y_coords = [pos[1] * 1e6 for pos in best_individual]

ax2.scatter(x_coords, y_coords, s=200, c='red', marker='o', 
           edgecolors='black', linewidths=2, alpha=0.7, label='Dots')
ax2.set_xlabel('x (µm)')
ax2.set_ylabel('y (µm)')
ax2.set_title(f'Best Configuration (Fitness = {best_fitness_final:.4f})')
ax2.set_aspect('equal')
ax2.grid(True, alpha=0.3)
ax2.legend()

# Add device boundaries as reference
ax2.axvline(-2.5, color='gray', linestyle='--', alpha=0.5, label='Device edge')
ax2.axvline(0.5, color='gray', linestyle='--', alpha=0.5)
ax2.axhline(-0.5, color='gray', linestyle='--', alpha=0.5)
ax2.axhline(0.5, color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig('optimization_results.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nFinal best fitness: {best_fitness_final:.6f}")
print(f"\nBest dot positions (µm):")
for i, (x, y) in enumerate(best_individual):
    print(f"  Dot {i+1}: ({x*1e6:.4f}, {y*1e6:.4f})")

In [ ]:
# Re-run simulation with best dot configuration and visualize FFT results
from simulation_worker import run_mumax3, build_magnetization_array, mag_tfft_select, measure_region_amplitude
from matplotlib.colors import ListedColormap
import matplotlib.cm as cm
from scipy.io import savemat

# Check if optimizer exists, if not load from checkpoint
if 'optimizer' not in globals():
    print("Optimizer not found in memory. Loading from checkpoint...")
    from genetic_optimizer import GeneticOptimizer
    import glob
    import re
    
    # Find all checkpoint files
    checkpoint_files = glob.glob('checkpoint_gen*.json')
    if not checkpoint_files:
        raise FileNotFoundError("No checkpoint files found. Please run the optimization first (cells 1-2).")
    
    # Extract generation numbers and sort numerically
    def get_generation_number(filename):
        match = re.search(r'checkpoint_gen(\d+)\.json', filename)
        return int(match.group(1)) if match else -1
    
    checkpoint_files.sort(key=get_generation_number)
    latest_checkpoint = checkpoint_files[-1]
    print(f"Loading: {latest_checkpoint}")
    
    optimizer = GeneticOptimizer(
        population_size=10,
        n_dots=8,
        x_range=(-2.4e-6, -0.1e-6),
        y_range=(-0.4e-6, 0.4e-6),
        mutation_rate=0.2,
        mutation_std=100e-9,
        crossover_rate=0.7,
        elite_fraction=0.2
    )
    optimizer.load_checkpoint(latest_checkpoint)
    print("Checkpoint loaded successfully!")

print("\nRe-running simulation with optimized dot positions...")

# Get best individual
best_dots, best_fitness = optimizer.get_best_individual()
print(f"\nBest fitness: {best_fitness:.6f}")
print(f"Dot positions:")
for i, (x, y) in enumerate(best_dots):
    print(f"  Dot {i+1}: ({x*1e6:.4f}, {y*1e6:.4f}) µm")

# Generate and run MuMax3 script
script = f"""
n := {len(best_dots)}

//  Mesh & sizes ---------------------------------
cell_size_x := 20e-9
cell_size_y := 20e-9
cell_size_z := 10e-9
grid_size_x := 250
grid_size_y := 50
grid_size_z := 20
SetGridsize(grid_size_x, grid_size_y, grid_size_z)
SetCellsize(cell_size_x, cell_size_y, cell_size_z)

//  Geometry from PNG and regions from OVF --------
//  Geometry from PNG (white=inside, black=outside)
device_geom := (ImageShape("mumax_geometry.png")).sub(cuboid(cell_size_x*grid_size_x,cell_size_y*grid_size_y,cell_size_z*grid_size_z/2).transl(0, 0, cell_size_z*grid_size_z/4))

regions.LoadFile("regions_map.ovf")
saveas(regions, "regions_map")

//  Material parameters and alpha modulation -------
Msat = 1.4e5
Aex  = 3.5e-12
edge_alpha := 0.5
device_alpha := 2e-4
alpha = edge_alpha
k := -5.0
for i:=1; i<254; i+=1{{
    s := i/253
    new_alpha := edge_alpha + (device_alpha - edge_alpha) * ((exp(k*s) - 1) / (exp(k) - 1))
    alpha.setRegion(i, new_alpha)
}}
saveas(alpha, "alpha_map")
//  Dots above device as region 254 ----------------
dots := cylinder(0, 0)
if n != 0 {{
    {chr(10).join([f"dots = dots.add((cylinder(100e-9,100e-9).transl({x:.15e}, {y:.15e}, 0)).transl(0, 0, 50e-9))" for x, y in best_dots]).replace(chr(10), chr(10) + "    ")}
    defregion(254, dots)
    Msat.setRegion(254, 1.145e6)
    Aex.setRegion(254, 7.5e-12)
    alpha.setRegion(254, 0.2)
}} 

SetGeom(device_geom.add(dots))
saveas(geom, "geom")

//  Simulation parameters -------------------------
T = 50e-9
f1 := 2.6e9
f2 := 2.8e9
sample_dt := 50e-12
m = uniform(0.02, 0.02, 1)
B_ext = vector(0, 0, 0.2)
autosave(m,sample_dt)
TableAutosave(sample_dt)

//  Input stripline (region 255) -------------------
input_stripline := cuboid(300e-9, 0.8e-6, 100e-9).transl(-2.5e-6 + 150e-9, 0, -50e-9)
defregion(255,input_stripline)
B_ext.setregion(255, vector(0, (0.1e-3)*sin(2*pi*f1*t) + (0.1e-3)*sin(2*pi*f2*t), 0.2))

//  Run -------------------------------------------
run(T)
"""

# Run simulation
table, fields = run_mumax3(script, name='best_configuration', verbose=True)
savemat("output.mat",fields,appendmat=True)
print("\nSimulation complete! Building magnetization array and performing FFT...")

# Build 5D magnetization array
M, timestamps = build_magnetization_array(fields, pattern='m')
print(f"Magnetization array shape: {M.shape}")

# Extract geometry for dot overlay
geom = fields['geom']
# Geometry is 4D: (component, z, y, x) - take first component (region labels)
if geom.ndim == 4:
    geom = geom[0, :, :, :]  # Extract region labels
nz, ny, nx = geom.shape

# Get physical coordinates
device_size_x = 5e-6
device_size_y = 1e-6
x_phys = np.linspace(-device_size_x/2, device_size_x/2, nx)
y_phys = np.linspace(-device_size_y/2, device_size_y/2, ny)

# Extract dots from upper half of geometry
dots_geom = geom[nz//2:, :, :]
dots_map = (dots_geom == 254).any(axis=0)

# Perform FFT analysis on y component (excited by stripline)
dt = 50e-12
freq_targets = [2.6e9, 2.8e9]
maps, fpos = mag_tfft_select(M, component='y', dt=dt, fsel=freq_targets, 
                             dimorder='zyx', detrend=True, window='hann', stat='amp')

print(f"FFT complete. Extracted frequencies: {fpos/1e9} GHz")

# Measure output amplitudes
measurement_size = 300e-9
output_top_center = (1.5e-6, 0.35e-6)
output_bottom_center = (1.5e-6, -0.35e-6)

results = {}
for i, (fmap, f) in enumerate(zip(maps, fpos)):
    freq_label = f"{f/1e9:.2f} GHz"
    
    amp_top, bbox_top = measure_region_amplitude(
        fmap, output_top_center[0], output_top_center[1], 
        measurement_size, device_size_x, device_size_y
    )
    
    amp_bottom, bbox_bottom = measure_region_amplitude(
        fmap, output_bottom_center[0], output_bottom_center[1],
        measurement_size, device_size_x, device_size_y
    )
    
    results[freq_label] = {
        'top': amp_top, 
        'bottom': amp_bottom,
        'bbox_top': bbox_top,
        'bbox_bottom': bbox_bottom
    }

# Print results
print("\n" + "="*60)
print("OPTIMIZED CONFIGURATION RESULTS")
print("="*60)
for freq_label, data in results.items():
    print(f"\n{freq_label}:")
    print(f"  Top corridor:    {data['top']:.6e}")
    print(f"  Bottom corridor: {data['bottom']:.6e}")
    print(f"  Ratio (T/B):     {data['top']/data['bottom']:.4f}")

f1_label = f"{fpos[0]/1e9:.2f} GHz"
f2_label = f"{fpos[1]/1e9:.2f} GHz"
selectivity_top = results[f1_label]['top'] / (results[f2_label]['top'] + 1e-10)
selectivity_bottom = results[f2_label]['bottom'] / (results[f1_label]['bottom'] + 1e-10)
print(f"\nSelectivity (2.6 GHz → Top):    {selectivity_top:.4f}")
print(f"Selectivity (2.8 GHz → Bottom): {selectivity_bottom:.4f}")
print(f"Combined fitness score:         {selectivity_top * selectivity_bottom:.4f}")

# Create turbo colormap
turbo_data = cm.get_cmap('turbo', 256)(np.linspace(0, 1, 256))
turbo_cmap = ListedColormap(turbo_data)


In [ ]:
# Visualize FFT results with dot positions overlaid
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for i, (fmap, f) in enumerate(zip(maps, fpos)):
    ax = axes[i]
    
    # Sum over z-dimension for 2D visualization
    amp_map = np.sum(fmap, axis=0)
    
    # Plot with turbo colormap and physical coordinates
    extent = [x_phys[0]*1e6, x_phys[-1]*1e6, y_phys[0]*1e6, y_phys[-1]*1e6]
    im = ax.imshow(amp_map, origin='lower', cmap=turbo_cmap, 
                   extent=extent, aspect='auto', interpolation='bilinear')
    
    # Overlay dot contours from geometry
    X, Y = np.meshgrid(x_phys, y_phys)
    ax.contour(X*1e6, Y*1e6, dots_map, levels=[0.5], colors='white', 
               linewidths=1.5, linestyles='-', alpha=0.8)
    
    # Overlay dot positions as circles
    for dot_x, dot_y in best_dots:
        circle = plt.Circle((dot_x*1e6, dot_y*1e6), 0.05, 
                          color='white', fill=False, linewidth=2, 
                          linestyle='--', alpha=0.9)
        ax.add_patch(circle)
    
    # Draw measurement boxes
    freq_label = f"{f/1e9:.2f} GHz"
    bbox_top = results[freq_label]['bbox_top']
    bbox_bottom = results[freq_label]['bbox_bottom']
    
    # Top measurement box
    x_box_top = [x_phys[bbox_top[0]]*1e6, x_phys[bbox_top[1]]*1e6]
    y_box_top = [y_phys[bbox_top[2]]*1e6, y_phys[bbox_top[3]]*1e6]
    ax.plot([x_box_top[0], x_box_top[1], x_box_top[1], x_box_top[0], x_box_top[0]],
            [y_box_top[0], y_box_top[0], y_box_top[1], y_box_top[1], y_box_top[0]],
            'lime', linewidth=2, linestyle='--', label='Measurement region')
    
    # Bottom measurement box
    x_box_bottom = [x_phys[bbox_bottom[0]]*1e6, x_phys[bbox_bottom[1]]*1e6]
    y_box_bottom = [y_phys[bbox_bottom[2]]*1e6, y_phys[bbox_bottom[3]]*1e6]
    ax.plot([x_box_bottom[0], x_box_bottom[1], x_box_bottom[1], x_box_bottom[0], x_box_bottom[0]],
            [y_box_bottom[0], y_box_bottom[0], y_box_bottom[1], y_box_bottom[1], y_box_bottom[0]],
            'lime', linewidth=2, linestyle='--')
    
    # Formatting
    ax.set_xlabel('x (µm)', fontsize=12)
    ax.set_ylabel('y (µm)', fontsize=12)
    ax.set_title(f'{f/1e9:.2f} GHz - my amplitude', fontsize=13, fontweight='bold')
    ax.set_aspect('equal')
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label('Amplitude', fontsize=10)
    
    # Add text annotations with measurements
    amp_top = results[freq_label]['top']
    amp_bottom = results[freq_label]['bottom']
    ax.text(0.02, 0.98, f'Top: {amp_top:.2e}\nBottom: {amp_bottom:.2e}\nRatio: {amp_top/amp_bottom:.2f}',
            transform=ax.transAxes, fontsize=10, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.savefig('best_configuration_fft.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nVisualization complete!")


In [ ]:
savemat("output.mat",fields,appendmat=True)